In [2]:
import pandas as pd
import pennylane as qml
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize as opt
import sklearn as sk
import pathlib as Path

In [3]:
np.random.seed(42)

In [4]:
# Read the data from a CSV file from data/EstData.csv
data = pd.read_csv("../../data/EstData.csv")
data

,ID,BW,COMED,DOSE,TIME,DV,EVID,MDV,AMT,CMT,DVID
0,1,58,0,0,0,18.617400,0,0,0,3,2
1,1,58,0,0,1,13.778300,0,0,0,3,2
2,1,58,0,0,2,16.574700,0,0,0,3,2
3,1,58,0,0,4,16.848600,0,0,0,3,2
4,1,58,0,0,8,18.645900,0,0,0,3,2
...,...,...,...,...,...,...,...,...,...,...,...
2815,48,61,0,10,840,3.377700,0,0,0,3,2
2816,48,61,0,10,1008,1.548160,0,0,0,2,1
2817,48,61,0,10,1008,3.438450,0,0,0,3,2
2818,48,61,0,10,1176,0.689746,0,0,0,2,1



### Overview of the data ###

Pharmacokinetics (PK): describes how the body interacts with the drug in terms of its absorption, distribution, metabolism, and elimination. It models how drug concentration changes over time in the body
Pharmacodynamics (PD): describes how the drug interacts with the body, linking drug concentration to its biological effect. PD models relate the concentration of a drug to the intensity and duration of its effects.

PK/PD: A combined PK/PD model integrates both aspects; it predicts the time course of drug effects based on the dosing regimen.

1.) ID: Subject Identifier

2.) BW: Body Weight (in kg)

3.) COMED:  Concomitant medication indicator for the subject (0: No, 1: Yes)

4.) DOSE: Dose level in mg. This column represents the amount of drug administered to the subject. It is typically measured in units like milligrams (mg) or micrograms (μg). The dose is a critical factor in determining the drug’s pharmacokinetic and pharmacodynamic properties.

5.) TIME: Time in hours. Indicates the time elapsed since the start of the first drug administration. 
                         Time is typically measured in hours or minutes and is essential for plotting concentration-time profiles.

6.) DV (Dependent Variable): Compound concentration (mg/L) for DVID=1. Biomarker level (ng/mL) for DVID=2. 
                    This column usually represents observed data, such as drug concentration in plasma or another biological matrix. 
                    It can also refer to the measurement of a biomarker or response variable affected by the drug, such as blood pressure or heart rate.

7.) EVID (Event ID): This is an event identifier used in NONMEM (a common software in pharmacometrics). It signifies the type of event occurring:
        EVID = 0 for observation events (e.g., a concentration measurement),
        EVID = 1 for dosing events
        
8.) MDV (Missing Dependent Variable): This value indicates whether the dependent variable (DV) is missing. An MDV of 1 means the DV value is missing, while 0 means it is present

9.) AMT (Amount): Stands for the actual dose amount administered, especially applicable during infusion dosing. AMT will be zero for observation records.

10.) CMT (Compartment): This denotes the compartment where the event (e.g., dosing or sampling) occurs. In PK models, different compartments (like central and peripheral) help to describe drug kinetics.

11.) DVID (Dependent Variable ID): This identifier helps distinguish between different types of DVs in the dataset. For example, you might have multiple measurement types such as concentrations and biomarkers.

